### Dependencies

In [57]:
def get_season(month):
    if month in [12,1,2]:
        return 1
    if month in [3,4,5]:
        return 2
    if month in [6,7,8]:
        return 3
    if month in [9,10,11]:
        return 4

def get_am(hour):
    if hour in range(0,12):
        return 1
    else:
        return 0
    
def prepare_x_test(measurement, forecast, past_n_steps, pred_period):
    
    #concat past n steps from measurement 
    df = measurement.set_index('datetime')
    df.fillna(method='ffill')
    df=proc.get_past_n_steps(df, past_n_steps)

    #calculate forecast_time
    df['forecast_time'] = df['present_time']+timedelta(hours=pred_period)

    #join forecast according to forecast time 
    forecast = forecast.set_index('f_date') 
    forecast = forecast.add_suffix('_forecast')
    df = pd.merge(df, forecast, how = 'left', left_on = 'forecast_time', right_on ='f_date')
    #add cos day
    df = proc.smooth_day_hour(df)
    #fill missing forecasts as 0
    df.fillna(value=0, inplace=True) 
    #keep first row 
    df = df.head(1)
    #drop timestamp columns
    df_out = df.drop(['present_time','forecast_time'], axis=1)
    return df_out

# test_df = prepare_x_test(measurement, forecast, past_steps, predict )

    
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from datetime import timedelta
import pickle

from utils import utils_scenario as utils, data_preparation as prep, data_process as proc

### Load data 
 - Load forecast data
 - Fix prediction date
 - Load last 48 hours measurement data

In [2]:
# forecast data
forecast = pd.read_csv('./data/last_forecast.csv')
forecast['p_date'] = pd.to_datetime(forecast['p_date'],format='%Y-%m-%d %H:%M:%S')
forecast['f_date'] = pd.to_datetime(forecast['f_date'],format='%Y-%m-%d %H:%M:%S')
forecast.drop(columns=['f_day','f_hour','p_date','f_period'],inplace=True)
forecast.head()

,f_date,speed,angle,temp,rad,cos_wind_dir,sin_wind_dir
0,2020-06-01 00:00:00,3.8,229,19.3,0,-0.656059,-0.754710
1,2020-06-01 01:00:00,3.2,224,19.2,0,-0.719340,-0.694658
2,2020-06-01 02:00:00,3.1,220,19.2,0,-0.766044,-0.642788
3,2020-06-01 03:00:00,3.0,216,19.1,0,-0.809017,-0.587785
4,2020-06-01 04:00:00,2.6,212,19.1,0,-0.848048,-0.529919


### Model
- Prepare data 
- Make prediction 
- Join with next 48h measurement 
- Save input and result 

In [3]:
result  = pd.DataFrame(columns=['past_n_steps','pred_period','speed', 'cos_wind_dir','sin_wind_dir']) 
pred_list = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48]
past_n_steps = 48


for i in range(0,19):
    # prediction date 
    date_to_predict = pd.to_datetime('2020-07-07 12:00:00',format='%Y-%m-%d %H:%M:%S') + timedelta(hours=i)
    # get last 48 hours measurement 
    all_measurement = pd.read_csv('./data/processed_measurement.csv')
    all_measurement['datetime'] = pd.to_datetime(all_measurement['datetime'],format='%Y-%m-%d %H:%M:%S')

    measurement = all_measurement.loc[(all_measurement.datetime<=date_to_predict) \
                        & (all_measurement.datetime>=(date_to_predict - timedelta(hours=48))),]
    measurement.drop(columns='wind_dir',inplace=True)


    for pred in pred_list: 
        #prepare data to be the same format as training data 
        x_test = prepare_x_test(measurement, forecast, past_n_steps, pred)
        x_test= np.array(x_test) #change to array 

        #read 3 models for speed, cos_wind, sin_wind
        xgb1= pickle.load(open('trained_models_04072020//speed_t_'+str(pred), 'rb'))
        xgb2 = pickle.load(open('trained_models_04072020//cos_wind_dir_t_'+str(pred), 'rb'))
        xgb3 = pickle.load(open('trained_models_04072020//sin_wind_dir_t_'+str(pred), 'rb'))

        #predict 
        speed = xgb1.predict(x_test)[0]
        cos_wind = xgb2.predict(x_test)[0]
        sin_wind = xgb3.predict(x_test)[0]

        #record accuracy
        result = result.append({'past_n_steps': str(past_n_steps),
                                'pred_period': str(pred),
                                'speed':round(speed,3),
                                'cos_wind_dir':cos_wind,
                                'sin_wind_dir':sin_wind}, ignore_index=True)    

    #convert cos and sin to wind_dir:
    result['wind_dir'] = result.apply(lambda row: utils.get_angle_in_degree(row['cos_wind_dir'],row['sin_wind_dir']),axis = 1)
    result['datetime'] = date_to_predict + result['pred_period'].map(lambda x : timedelta(hours=int(x)))

    # get next 48hours measurement
    next_measurement = all_measurement.loc[(all_measurement.datetime>date_to_predict) \
                        & (all_measurement.datetime<=(date_to_predict + timedelta(hours=48))),]
    next_measurement['wind_dir'] = next_measurement.apply(lambda row: utils.get_angle_in_degree(row['cos_wind_dir'],row['sin_wind_dir']),axis = 1)

    # merge with results
    result_vs_measurement = result[['datetime','speed','wind_dir']].merge(next_measurement[['datetime','speed','wind_dir']] \
    .rename(columns={'speed':'real_speed','wind_dir':'real_wind_dir'}))

    # append input measurement
    measurement = all_measurement.loc[(all_measurement.datetime<=date_to_predict) \
                        & (all_measurement.datetime>=(date_to_predict - timedelta(hours=48))),]
    measurement = measurement[['datetime','speed','wind_dir']] \
    .rename(columns={'speed':'real_speed','wind_dir':'real_wind_dir'})

    # save file
    (pd.concat([measurement,result_vs_measurement],axis=0).reset_index(drop=True)) \
    .to_excel('./data/result/test_' + str(date_to_predict)[0:13].replace(' ','_') + '.xlsx')
    print('./data/result/test_' + str(date_to_predict)[0:13].replace(' ','_') + '.xlsx')

cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
./data/result/test_2020-07-07_12.xlsx
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
./data/result/test_2020-07-07_13.xlsx
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0

cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
c

./data/result/test_2020-07-08_00.xlsx
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0


./data/result/test_2020-07-08_03.xlsx
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0
cos and sin out of range, returned 0


./data/result/test_2020-07-08_06.xlsx


In [1]:
#------ Cynthia Edit ---- # 

In [8]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from datetime import timedelta
import pickle

from utils import utils_scenario as utils, data_preparation as prep, data_process as proc

In [69]:
measurement_format = pd.read_csv("data/measurement_format.csv") 
forecast_format = pd.read_csv("data/forecast_format.csv") 

In [70]:
# forecast data
forecast = pd.read_csv('./data/last_forecast.csv')
forecast['p_date'] = pd.to_datetime(forecast['p_date'],format='%Y-%m-%d %H:%M:%S')
forecast['f_date'] = pd.to_datetime(forecast['f_date'],format='%Y-%m-%d %H:%M:%S')
forecast.drop(columns=['f_day','f_hour','p_date','f_period'],inplace=True)
forecast.head()
forecast['wind_dir_reconstructed'] = forecast.apply(lambda row: utils.get_angle_in_degree(row['cos_wind_dir'],row['sin_wind_dir']),axis = 1)
print('reconstructed wind_dir using current cos and sin')
forecast.head()

reconstructed wind_dir using current cos and sin


,f_date,speed,angle,temp,rad,cos_wind_dir,sin_wind_dir,wind_dir_reconstructed
0,2020-06-01 00:00:00,3.8,229,19.3,0,-0.656059,-0.754710,311.0
1,2020-06-01 01:00:00,3.2,224,19.2,0,-0.719340,-0.694658,316.0
2,2020-06-01 02:00:00,3.1,220,19.2,0,-0.766044,-0.642788,320.0
3,2020-06-01 03:00:00,3.0,216,19.1,0,-0.809017,-0.587785,324.0
4,2020-06-01 04:00:00,2.6,212,19.1,0,-0.848048,-0.529919,328.0


### Issue 1: smooth wind direction:
- the cos and sin of wind_dir seems to be wrong, re-constructed wind_dir does not agree with the real wind_dir
- the same issue could arise with the forecast processing, but we don't have the real wind_dir for forecast to check. 
- you can use the 'smooth_wind_dir' function in data_preperation.py file to correct this

In [74]:
date_to_predict = pd.to_datetime('2020-07-07 12:00:00',format='%Y-%m-%d %H:%M:%S')
# get last 48 hours measurement 
all_measurement = pd.read_csv('./data/processed_measurement.csv')
all_measurement['datetime'] = pd.to_datetime(all_measurement['datetime'],format='%Y-%m-%d %H:%M:%S')

measurement = all_measurement.loc[(all_measurement.datetime<=date_to_predict) \
                    & (all_measurement.datetime>=(date_to_predict - timedelta(hours=48))),]
# measurement.drop(columns='wind_dir',inplace=True)
#convert cos and sin to wind_dir:
measurement['wind_dir_reconstructed'] = measurement.apply(lambda row: utils.get_angle_in_degree(row['cos_wind_dir'],row['sin_wind_dir']),axis = 1)
print('reconstructed wind_dir using current cos and sin')
print(measurement[['wind_dir_reconstructed','wind_dir']].head(5))

#instead, pleas use 
measurement = prep.smooth_wind_dir(measurement)
print('using smooth_wind_dir to smooth functions')
print(measurement.head())

reconstructed wind_dir using current cos and sin
     wind_dir_reconstructed    wind_dir
107               49.145660   52.041667
108              224.833358  295.106667
109              218.301170  272.433333
110              215.627484  211.701667
111              226.018445  314.845000
smooth wind direction
using smooth_wind_dir to smooth functions
               datetime  cos_wind_dir  sin_wind_dir     speed  temp  \
107 2020-07-05 12:00:00      0.615088      0.788458  4.223333   NaN   
108 2020-07-05 13:00:00      0.424305     -0.905519  4.278333   NaN   
109 2020-07-05 14:00:00      0.042457     -0.999098  5.103333   NaN   
110 2020-07-05 15:00:00     -0.850796     -0.525496  5.020000   NaN   
111 2020-07-05 16:00:00      0.705191     -0.709017  5.300000   NaN   

      radiation  precip  season  wind_dir_reconstructed  
107  650.900000     NaN       3               49.145660  
108  748.566667     NaN       3              224.833358  
109  780.166667     NaN       3              2

### Issue 2: column order need to be the same as training df
- for both forecast and measurement data, the column order do not agree with the training format
- further more, the precipitation feature is abscent in forecast df, we fill with zero for now

In [75]:
measurement = measurement[measurement_format.columns]
print(measurement.head())
forecast['precip']=0
forecast = forecast[forecast_format.columns]
print(forecast.head())

               datetime     speed  cos_wind_dir  sin_wind_dir  temp  \
107 2020-07-05 12:00:00  4.223333      0.615088      0.788458   NaN   
108 2020-07-05 13:00:00  4.278333      0.424305     -0.905519   NaN   
109 2020-07-05 14:00:00  5.103333      0.042457     -0.999098   NaN   
110 2020-07-05 15:00:00  5.020000     -0.850796     -0.525496   NaN   
111 2020-07-05 16:00:00  5.300000      0.705191     -0.709017   NaN   

      radiation  precip  season  
107  650.900000     NaN       3  
108  748.566667     NaN       3  
109  780.166667     NaN       3  
110  782.783333     NaN       3  
111  721.450000     NaN       3  
               f_date  speed  temp  rad  precip  cos_wind_dir  sin_wind_dir
0 2020-06-01 00:00:00    3.8  19.3    0       0     -0.656059     -0.754710
1 2020-06-01 01:00:00    3.2  19.2    0       0     -0.719340     -0.694658
2 2020-06-01 02:00:00    3.1  19.2    0       0     -0.766044     -0.642788
3 2020-06-01 03:00:00    3.0  19.1    0       0     -0.809017    

### Issue 3: Temperature features is missing
- Temperature is an important feature
- Need to fill missing with zero

In [76]:
measurement = measurement.fillna(0)
measurement.head()

,datetime,speed,cos_wind_dir,sin_wind_dir,temp,radiation,precip,season
107,2020-07-05 12:00:00,4.223333,0.615088,0.788458,0.0,650.900000,0.0,3
108,2020-07-05 13:00:00,4.278333,0.424305,-0.905519,0.0,748.566667,0.0,3
109,2020-07-05 14:00:00,5.103333,0.042457,-0.999098,0.0,780.166667,0.0,3
110,2020-07-05 15:00:00,5.020000,-0.850796,-0.525496,0.0,782.783333,0.0,3
111,2020-07-05 16:00:00,5.300000,0.705191,-0.709017,0.0,721.450000,0.0,3


### Now try to do prediction again

In [ ]:
result  = pd.DataFrame(columns=['past_n_steps','pred_period','speed', 'cos_wind_dir','sin_wind_dir']) 
pred_list = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48]
past_n_steps = 48

#get forecast 
forecast = pd.read_csv('./data/last_forecast.csv')
forecast['p_date'] = pd.to_datetime(forecast['p_date'],format='%Y-%m-%d %H:%M:%S')
forecast['f_date'] = pd.to_datetime(forecast['f_date'],format='%Y-%m-%d %H:%M:%S')
forecast.drop(columns=['f_day','f_hour','p_date','f_period'],inplace=True)
#match orders
forecast['precip']=0
forecast = forecast[forecast_format.columns]
forecast = forecast.fillna(0)


for i in range(0,19):
    # prediction date 
    date_to_predict = pd.to_datetime('2020-07-07 12:00:00',format='%Y-%m-%d %H:%M:%S') + timedelta(hours=i)
    # get last 48 hours measurement 
    all_measurement = pd.read_csv('./data/processed_measurement.csv')
    all_measurement['datetime'] = pd.to_datetime(all_measurement['datetime'],format='%Y-%m-%d %H:%M:%S')

    measurement = all_measurement.loc[(all_measurement.datetime<=date_to_predict) \
                        & (all_measurement.datetime>=(date_to_predict - timedelta(hours=48))),]
    
    #Fix issue 1:  smooth wind direction
    measurement = prep.smooth_wind_dir(measurement) 
    #Fix issue 2:  same column order
    measurement = measurement[measurement_format.columns]
    #Fix issue 3:  fill na
    measurement = measurement.fillna(0)

    for pred in pred_list: 
        #prepare data to be the same format as training data 
        x_test = prepare_x_test(measurement, forecast, past_n_steps, pred)
        x_test= np.array(x_test) #change to array 

        #read 3 models for speed, cos_wind, sin_wind
        xgb1= pickle.load(open('../trained_models/speed_t_'+str(pred), 'rb'))
        xgb2 = pickle.load(open('../trained_models/cos_wind_dir_t_'+str(pred), 'rb'))
        xgb3 = pickle.load(open('../trained_models/sin_wind_dir_t_'+str(pred), 'rb'))

        #predict 
        speed = xgb1.predict(x_test)[0]
        cos_wind = xgb2.predict(x_test)[0]
        sin_wind = xgb3.predict(x_test)[0]

        #record accuracy
        result = result.append({'past_n_steps': str(past_n_steps),
                                'pred_period': str(pred),
                                'speed':round(speed,3),
                                'cos_wind_dir':cos_wind,
                                'sin_wind_dir':sin_wind}, ignore_index=True)    

    #convert cos and sin to wind_dir:
    result['wind_dir'] = result.apply(lambda row: utils.get_angle_in_degree(row['cos_wind_dir'],row['sin_wind_dir']),axis = 1)
    result['datetime'] = date_to_predict + result['pred_period'].map(lambda x : timedelta(hours=int(x)))

    # get next 48hours measurement
    next_measurement = all_measurement.loc[(all_measurement.datetime>date_to_predict) \
                        & (all_measurement.datetime<=(date_to_predict + timedelta(hours=48))),]
    next_measurement['wind_dir'] = next_measurement.apply(lambda row: utils.get_angle_in_degree(row['cos_wind_dir'],row['sin_wind_dir']),axis = 1)

    # merge with results
    result_vs_measurement = result[['datetime','speed','wind_dir']].merge(next_measurement[['datetime','speed','wind_dir']] \
    .rename(columns={'speed':'real_speed','wind_dir':'real_wind_dir'}))

    # append input measurement
    measurement = all_measurement.loc[(all_measurement.datetime<=date_to_predict) \
                        & (all_measurement.datetime>=(date_to_predict - timedelta(hours=48))),]
    measurement = measurement[['datetime','speed','wind_dir']] \
    .rename(columns={'speed':'real_speed','wind_dir':'real_wind_dir'})

    # save file
    (pd.concat([measurement,result_vs_measurement],axis=0).reset_index(drop=True)) \
    .to_excel('./data/result/test2_' + str(date_to_predict)[0:13].replace(' ','_') + '.xlsx')
    print('./data/result/test2_' + str(date_to_predict)[0:13].replace(' ','_') + '.xlsx')

smooth wind direction
./data/result/test2_2020-07-07_12.xlsx
smooth wind direction
./data/result/test2_2020-07-07_13.xlsx
smooth wind direction
